# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

financial_trading_crew = Crew(
    agents = [data_analyst_agent, trading_strategy_agent, execution_agent, risk_management_agent],
    tasks = [data_analysis_task, strategy_development_task, execution_planning_task, risk_assessment_task],
    manager_llm = ChatOpenAI(model = 'gpt-3.5-turbo', temperature = 0.7),
    process = Process.hierarchical,
    verbose = True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate tasks to the right team members in order to continuously monitor and analyze market data for AAPL effectively.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning",
    "context": "The task involves collecting, analyzing, and interpreting market data for AAPL on a regular basis. Utilize statistical modeling and machine learning techniques to identify trends and predict market movements."
}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL and analyze it using statistical modeling and machine learning techniques

Final Answer: 
After analyzing the market data for AAPL from Yahoo Finance, the stock price for Apple Inc. (AAPL) is currently at $217.96 with a slight increase of 0.22%. The market cap is reported to be 3.342T, and the company overview highlights its products and services in the consumer electronics industry. Additionally, key financial metrics and performance comparisons with industry peers are provided. The stock has shown positive returns over different time periods, and various valuation measures and profitability indicators are detailed. Lastly, the research analysis, company insights, and related companies to watch are also included in the report.

> Finished chain.
 

After analyzing the market data for AAPL from Yahoo Finance, the stock price for Apple Inc. (AAPL) is currently at $217.96 with a slight increase of 0.22%. The market cap is reported to be 3.342T, and the company overview highlights its products and services in the consumer electronics industry. Additionally, key 

Final Answer: 
Based on the analysis of market data for AAPL from Yahoo Finance, some potential growth opportunities for Apple Inc. include its strong performance in the consumer electronics industry, consistent revenue generation, and a solid financial position with a market cap of 3.34 trillion. Additionally, the company's profitability metrics such as a profit margin of 26.31% and a return on equity of 147.25% indicate positive growth potential. However, threats to AAPL may arise from factors such as increasing competition in the industry, market fluctuations, and potential regulatory challenges. Overall, AAPL shows promise for growth but should be monitored closely for any emerging risks.

> Finished chain.
 

Based on the analysis of market data for AAPL from Yahoo Finance, some potential growth opportunities for Apple Inc. include its strong performance in the consumer electronics industry, consistent revenue generation, and a solid financial position with a market cap of 3.34 tr

Thought: I need to gather real-time market data for AAPL to provide accurate insights to my coworker.

Action: Read website content
Action Input: {"website_url": "https://www.marketwatch.com/investing/stock/aapl"} 

marketwatch.comPlease enable JS and disable any ad blocker

Action: Search the internet
Action Input: {"search_query": "AAPL current market conditions site:marketwatch.com"} 


Search results: Title: Apple Inc. Stock Quote (U.S.: Nasdaq) - AAPL
Link: https://www.marketwatch.com/investing/stock/aapl
Snippet: AAPL | Complete Apple Inc. stock news by MarketWatch. View real-time stock prices and stock quotes for a full financial overview.
---
Title: Apple Inc. Options - AAPL
Link: https://www.marketwatch.com/investing/stock/aapl/options
Snippet: Current price as of Jul 26, 2024 4:00 p.m.. 217.96. 220.00. 220.00. 4.10. -0.10. 4.00. 4.20. 21,735. 17,792. 220.00. 5.80. -0.60. 5.55. 5.95. 3,719. 7,291.
---
Title: Apple Inc. stock rises Monday, still underperforms market
Link: https

Final Answer: 
I was unable to access the website content related to AAPL's trading strategies as the page was not found. Therefore, I cannot provide insights on the potential risks associated with the proposed trading strategies for AAPL or recommend mitigation strategies at this time.

> Finished chain.
 

I was unable to access the website content related to AAPL's trading strategies as the page was not found. Therefore, I cannot provide insights on the potential risks associated with the proposed trading strategies for AAPL or recommend mitigation strategies at this time.

Final Answer:

Risk Analysis Report for the Proposed Trading Strategies and Execution Plans for AAPL:

After thorough analysis and consideration of the current market conditions and insights provided by the Data Analyst on AAPL's stock news and real-time prices, the following potential risks and mitigation strategies have been identified for the proposed trading strategies and execution plans for AAPL:

1. Market

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for the Proposed Trading Strategies and Execution Plans for AAPL:

After thorough analysis and consideration of the current market conditions and insights provided by the Data Analyst on AAPL's stock news and real-time prices, the following potential risks and mitigation strategies have been identified for the proposed trading strategies and execution plans for AAPL:

1. Market Risk:
The market risk associated with trading AAPL is primarily influenced by macroeconomic factors, industry trends, and overall market volatility. To mitigate this risk, it is essential to diversify the trading portfolio and closely monitor market indicators to make informed decisions.

2. Liquidity Risk:
AAPL's stock liquidity can be impacted by sudden market fluctuations or significant news events, leading to challenges in executing trades at desired prices. To address this risk, it is recommended to use limit orders and establish clear exit strategies to manage liquidity effectively.

3. Operational Risk:
Operational risks, such as technical failures, human errors, or regulatory changes, can disrupt trading activities and result in financial losses. Implementing robust risk management protocols, regular monitoring of trading systems, and conducting thorough compliance checks can help mitigate operational risks.

4. Regulatory Risk:
Regulatory changes or legal issues related to AAPL's industry sector can have a direct impact on trading activities and require compliance with specific guidelines. Staying informed about regulatory updates, maintaining transparency in trading practices, and seeking legal advice when necessary can help navigate regulatory risks effectively.

5. Reputational Risk:
Reputational risks arise from negative publicity, customer complaints, or unethical behavior, which can damage the credibility and trustworthiness of trading strategies. Building strong relationships with stakeholders, adhering to ethical standards, and prioritizing transparency in decision-making are crucial in managing reputational risks.

In conclusion, a comprehensive risk management approach that considers market dynamics, liquidity challenges, operational resilience, regulatory compliance, and reputational integrity is essential for successful trading strategies and execution plans for AAPL. By proactively identifying potential risks and implementing appropriate mitigation strategies, the team can optimize trading performance and achieve the user's day trading objectives effectively.